## Part 1 - Closest to (0,0,0) in the long run

In [1]:
import numpy as np

def load(file_name):
    file = open(file_name, 'r')
    poss = []
    vels = []
    accs = []
    for s in file:
        poss.append(tuple(int(n) for n in s[s.find('p=<')+3:s.find('>, v=<')].split(',')))
        vels.append(tuple(int(n) for n in s[s.find('v=<')+3:s.find('>, a=<')].split(',')))
        accs.append(tuple(int(n) for n in s[s.find('a=<')+3:s.rfind('>')].split(',')))
    return np.array(poss), np.array(vels), np.array(accs)

In [2]:
poss, vels, accs = load('in/day20.txt')
acc_mds = np.abs(accs).sum(axis=-1)  # Absolute values of accelerations of each particle
min_abs_acc = np.where(acc_mds == acc_mds.min())[0]  # Array of indexes with absolute acceleration = min
# If there were more than 1 such particles - initial velocity should have been considered.
print('Answer to part 1 - particle whose acceleration is the lowest = {}'.format(min_abs_acc))

Answer to part 1 - particle whose acceleration is the lowest = [150]


## Part 2 - How many particles collided?

In [3]:
p, v, a = np.array(poss), np.array(vels), np.array(accs)
print(f'Starting simulation with {len(p)} particles')
for t in range(100):
    v += a
    p += v
    unique, idx, inverse_idx, counts = np.unique(p, axis=0, return_index=True, return_inverse=True, return_counts=True)
    collision_mask = counts > 1
    if True in collision_mask:  # Collisions (non-unique positions) found
        collision_idx = idx[collision_mask]
        collision_poss = unique[collision_mask]  # Duplicated positions (collisions) to be removed
        collision_idx = [i for i in range(p.shape[0]) if p[i] in collision_poss]
        if t == 23:
            print(f'\n{[2358,-1602,-12] in collision_poss} {collision_poss}')
            print(f'\nt={t:02}, count={len(p)} {len(v)} {len(a)}, \
collisions:{collision_mask.sum()}\n{p[collision_idx]}\n\n{unique[collision_mask]}\n\n{counts}')
        p = np.delete(p, collision_idx, axis=0)
        v = np.delete(v, collision_idx, axis=0)
        a = np.delete(a, collision_idx, axis=0)
        print(f't={t:02}, after collisions remain {len(p)} {len(v)} {len(a)} particles')

print(f'\nPart 2 answer: {len(p)}')

Starting simulation with 1000 particles
t=09, after collisions remain 979 979 979 particles
t=10, after collisions remain 972 972 972 particles
t=11, after collisions remain 956 956 956 particles
t=12, after collisions remain 948 948 948 particles
t=13, after collisions remain 933 933 933 particles
t=15, after collisions remain 921 921 921 particles
t=16, after collisions remain 896 896 896 particles
t=17, after collisions remain 881 881 881 particles
t=19, after collisions remain 864 864 864 particles
t=20, after collisions remain 840 840 840 particles
t=22, after collisions remain 831 831 831 particles

True [[  0 -16 -12]]

t=23, count=831 831 831, collisions:1
[[    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [    0   -16   -12]
 [ 2358 -1602   -12]]

[[  0 -16 -12]]

[10  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 

In [4]:
ar = np.array([1,2,3,4,5,6,6,5,4,3,4,3])
print(np.unique(ar, return_counts=True, return_inverse=True))
print(np.searchsorted(ar, [1,2,6]))

(array([1, 2, 3, 4, 5, 6]), array([0, 1, 2, 3, 4, 5, 5, 4, 3, 2, 3, 2]), array([1, 1, 3, 3, 2, 2]))
[0 1 5]


### Part 2 answers
Too high: `772`.

Incorrect: `644`, `663`.

Correct: `657`

# Someone elses' solution
[Here](https://gist.github.com/GlenboLake/91fa9b990e46b6b624704e9a1c7495c8)

In [5]:
import re
from cmath import sqrt
from collections import defaultdict, namedtuple
from functools import reduce
from itertools import combinations
from time import time

Particle = namedtuple('Particle', ['pos', 'vel', 'acc'])

def parse_particle(line):
    pos_match = re.search('p=<(-?\d+),(-?\d+),(-?\d+)>', line)
    position = (int(pos_match.group(1)), int(pos_match.group(2)), int(pos_match.group(3)))
    vel_match = re.search('v=<(-?\d+),(-?\d+),(-?\d+)>', line)
    velocity = (int(vel_match.group(1)), int(vel_match.group(2)), int(vel_match.group(3)))
    acc_match = re.search('a=<(-?\d+),(-?\d+),(-?\d+)>', line)
    acceleration = (int(acc_match.group(1)), int(acc_match.group(2)), int(acc_match.group(3)))
    return Particle(position, velocity, acceleration)

def particle_at(particle, t):
    x = particle.pos[0] + particle.vel[0] * t + particle.acc[0] * t * (t + 1) // 2
    y = particle.pos[1] + particle.vel[1] * t + particle.acc[1] * t * (t + 1) // 2
    z = particle.pos[2] + particle.vel[2] * t + particle.acc[2] * t * (t + 1) // 2
    return x, y, z

def manhattan(point):
    return sum(map(abs, point))

def part1(particles):
    max_accel = max(sum(map(abs, p.acc)) for p in particles)
    return particles.index(min(particles, key=lambda p: manhattan(particle_at(p, 100 * max_accel))))

def will_collide(p1, p2):
    def is_int(c):
        return c.imag == 0 and (isinstance(c.real, int) or c.real.is_integer())

    def solve_quadratic(a, b, c):
        solutions = None
        if a:
            solutions = {(-b - sqrt(b ** 2 - 4 * a * c)) / (2 * a), (-b + sqrt(b ** 2 - 4 * a * c)) / (2 * a)}
        elif b:
            solutions = {-c / b}
        elif c:
            solutions = {c}
        if solutions is not None:
            solutions = set(map(lambda x: int(x.real), filter(is_int, solutions)))
        return solutions

    diff = Particle(tuple(a - b for a, b in zip(p1.pos, p2.pos)),
                    tuple(a - b for a, b in zip(p1.vel, p2.vel)),
                    tuple(a - b for a, b in zip(p1.acc, p2.acc)))
    tuples = [
        (diff.acc[0], diff.vel[0], diff.pos[0]),
        (diff.acc[1], diff.vel[1], diff.pos[1]),
        (diff.acc[2], diff.vel[2], diff.pos[2]),
    ]
    solutions = reduce(lambda a, b: a & b,
                       filter(lambda s: s is not None,
                              [solve_quadratic(a / 2, v + a / 2, p) for a, v, p in tuples]))

    if solutions:
        return min(s for s in solutions if s > 0)
    return None

def pairs_to_sets(data):
    items = {a for a, b in data} | {b for a, b in data}
    sets = []
    seen = set()
    for item in items:
        if item in seen:
            continue
        new_set = set()
        seen.add(item)
        for pair in data:
            if item in pair:
                seen.update(set(pair))
                new_set.update(set(pair))
        sets.append(new_set)
    return sets

def part2(particles):
    collisions = defaultdict(list)
    for a, b in combinations(particles, 2):
        t = will_collide(a, b)
        if t is not None:
            collisions[t].append({a, b})
    collisions = {k: pairs_to_sets(v) for k, v in collisions.items()}
    remaining = set(particles)
    for t, splosions in sorted(collisions.items()):
        for s in splosions:
            if len(remaining & s) > 1:
                remaining -= s
    return len(remaining)

if __name__ == '__main__':
    particles = [parse_particle(line) for line in open('in/day20.txt')]

    start = time()
    print('Part 1:', part1(particles))
    print('Part 2:', part2(particles))
    print(f'Took {time()-start}s')

Part 1: 150
Part 2: 657
Took 7.107012748718262s
